<a href="https://colab.research.google.com/github/alexnodejs/bandit/blob/main/HT_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [15]:
import pandas as pd
import pandas as pd
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

Дані медичне страхування

In [8]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/insurance.csv')

df['sex'] = df['sex'].map({'male': 0, 'female':1})
df['smoker'] = df['smoker'].map({'no': 0, 'yes':1})

df = df.drop(columns=['region'])

data = df[['age', 'bmi', 'expenses']]
df[['age', 'bmi', 'expenses']] = (data - data.min()) / (data.max() - data.min())

In [9]:
df.head()

,age,sex,bmi,children,smoker,expenses
0,0.021739,1,0.320755,0,1,0.251611
1,0.000000,0,0.479784,1,0,0.009636
2,0.217391,0,0.458221,3,0,0.053115
3,0.326087,0,0.180593,0,0,0.333010
4,0.304348,0,0.347709,0,0,0.043816


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   float64
 1   sex       1338 non-null   int64  
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   int64  
 5   expenses  1338 non-null   float64
dtypes: float64(3), int64(3)
memory usage: 62.8 KB


Потрібно на основі інформації людини спрогнозувати розмір її страховки `expenses`

# Завдання 1
Створіть клас Dataset

In [11]:
import torch
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, X, y):
      self.X = torch.FloatTensor(X.values).to(device)
      self.y = torch.FloatTensor(y.values).to(device)
      self.y = self.y.reshape(-1, 1)

    def __len__(self):
      return len(self.X)

    def __getitem__(self, idx):
      return self.X[idx], self.y[idx]

# Create dataset object
X = df.drop(columns=['expenses'])
y = df['expenses']

dataset = MyDataset(X, y)

## Перевірка

In [12]:

def check_initialization(X, y):
    try:
        dataset = MyDataset(X, y)
        if torch.is_tensor(dataset.X) and torch.is_tensor(dataset.y):
            print("[OK] Ініціалізація коректна.")
        else:
            print("[FAILED] Ініціалізація не вдалася: X або y не є тензорами.")
    except Exception as e:
        print(f"[FAILED] Помилка ініціалізації: {e}")

def check_y_shape(dataset):
    if dataset.y.ndim == 2 and dataset.y.shape[1] == 1:
        print("[OK] 'y' правильно перетворений на вектор-стовпець.")
    else:
        print(f"[FAILED] 'y' перетворений некоректно, поточна форма: {dataset.y.shape}")

def check_length(dataset, expected_length):
    if len(dataset) == expected_length:
        print("[OK] Перевірка довжини пройдена.")
    else:
        print(f"[FAILED] Перевірка довжини не вдалася. Очікувана довжина: {expected_length}, отримана: {len(dataset)}.")

def check_getitem(dataset, idx):
    try:
        X_item, y_item = dataset[idx]
        if torch.is_tensor(X_item) and torch.is_tensor(y_item):
            print(f"[OK] Отримання елемента коректне для індексу {idx}.")
        else:
            print(f"[FAILED] Отримання елемента некоректне для індексу {idx}.")
    except Exception as e:
        print(f"[FAILED] Помилка у методі `__getitem__`: {e}")

def check_tensor_types(dataset):
    if dataset.X.dtype == torch.float32 and dataset.y.dtype == torch.float32:
        print("[OK] X та y збережені як FloatTensor.")
    else:
        print(f"[FAILED] Невідповідність типу тензорів: X має тип {dataset.X.dtype}, y має тип {dataset.y.dtype}.")

In [13]:
def main_checks():
    print("\n=== Виконання перевірок ===")
    check_initialization(X, y)
    check_y_shape(dataset)
    check_length(dataset, len(X))
    check_getitem(dataset, 0)
    check_tensor_types(dataset)
    print("=== Перевірки завершені ===\n")


main_checks()


=== Виконання перевірок ===
[OK] Ініціалізація коректна.
[OK] 'y' правильно перетворений на вектор-стовпець.
[OK] Перевірка довжини пройдена.
[OK] Отримання елемента коректне для індексу 0.
[OK] X та y збережені як FloatTensor.
=== Перевірки завершені ===



# Завдання 2
Створіть DataLoader. Перевірте розміри даних в першому пакеті. Розмір пакетів оберіть самостійно

In [16]:
batch_size = 64
dataloader = DataLoader(dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        generator=torch.Generator(device=device))

## Перевірка

In [17]:

def check_batch_size(batch_size):
    if isinstance(batch_size, int) and batch_size > 0:
        print(f"[OK] batch_size встановлений як {batch_size}.")
    else:
        print(f"[FAILED] batch_size некоректний, поточне значення: {batch_size}")

def check_dataloader_initialization(dataset, batch_size):
    try:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        if isinstance(dataloader, DataLoader):
            print(f"[OK] DataLoader створений коректно з batch_size = {batch_size}.")
        else:
            print("[FAILED] DataLoader не створений.")
    except Exception as e:
        print(f"[FAILED] Помилка під час ініціалізації DataLoader: {e}")

def check_dataloader_loading(dataloader, batch_size):
    try:
        for batch in dataloader:
            print(f"[OK] Форма ознак у батчі: {batch[0].shape}, Форма цільових змінних: {batch[1].shape}")
            if batch[0].shape[0] == batch_size or len(dataloader.dataset) % batch_size != 0:
                print(f"[OK] DataLoader завантажує дані пакетами по {batch_size}.")
            else:
                print(f"[FAILED] Розмір батчу некоректний, очікувано: {batch_size}")
            break
    except Exception as e:
        print(f"[FAILED] Помилка під час завантаження DataLoader: {e}")

# Main function to run all checks
def main_checks_dataloader(X, y, batch_size):
    # Create an instance of the dataset
    dataset = MyDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    print("\n=== Виконання перевірок DataLoader ===")
    check_batch_size(batch_size)
    check_dataloader_initialization(dataset, batch_size)
    check_dataloader_loading(dataloader, batch_size)
    print("=== Перевірки завершені ===\n")

In [18]:
def main_checks_dataloader():
    print("\n=== Виконання перевірок DataLoader ===")
    check_batch_size(batch_size)
    check_dataloader_initialization(dataset, batch_size)
    check_dataloader_loading(dataloader, batch_size)
    print("=== Перевірки завершені ===\n")

main_checks_dataloader()


=== Виконання перевірок DataLoader ===
[OK] batch_size встановлений як 64.
[OK] DataLoader створений коректно з batch_size = 64.
[OK] Форма ознак у батчі: torch.Size([64, 5]), Форма цільових змінних: torch.Size([64, 1])
[OK] DataLoader завантажує дані пакетами по 64.
=== Перевірки завершені ===



In [19]:
X_batch, y_batch = next(iter(dataloader))


In [20]:
X_batch.shape

torch.Size([64, 5])

In [21]:
y_batch.shape

torch.Size([64, 1])

# Завдання 3
Створіть нейромережу:
* Кількість шарів оберіть між 1-5
* Виберіть різні функції активації: Relu, LeakyRelu, Swish
* Кілкість нейронів у прихованих шарах повинна **не зростати**

In [24]:
from torch import nn
import pandas as pd

# model = nn.Sequential( # experiment with other models
#     nn.Linear(5, 5),
#     nn.ReLU(),
#     nn.Linear(5, 3),
#     nn.LeakyReLU(),
#     nn.Linear(3, 1)
# )

model = nn.Sequential(
    nn.Linear(5, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 4),
    nn.LeakyReLU(),
    nn.Linear(4, 3),
    nn.LeakyReLU(),
    nn.Linear(3, 2),
    nn.LeakyReLU(),
    nn.Linear(2, 1)
)

model

Sequential(
  (0): Linear(in_features=5, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=5, out_features=4, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=4, out_features=3, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=3, out_features=2, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=2, out_features=1, bias=True)
)

## Перевірка

In [25]:
def check_layers(model):
    # Перевірка кількості шарів
    num_layers = len(list(model.children()))
    if 1 <= num_layers <= 5:
        print("[OK] Кількість шарів: ", num_layers)
    else:
        print("[FAILED] Кількість шарів: ", num_layers)

def check_activation_functions(model):
    # Перевірка наявності правильних функцій активації
    valid_activations = [nn.ReLU, nn.LeakyReLU, nn.SiLU]  # SiLU is еквівалент Swish
    layers = list(model.children())

    # Перевірка першого шару
    first_layer = layers[0]
    if isinstance(first_layer, nn.Linear) and first_layer.in_features == dataset.X.shape[1]:
        print(f"[OK] Кількість вхідних нейронів першого шару відповідає {dataset.X.shape[1]}")
    else:
        print("[FAILED] Кількість вхідних нейронів першого шару: ", first_layer.in_features,
              " (очікується: ", dataset.X.shape[1], ")")

    for layer in layers[1:]:  # Починаємо з другого шару
        if isinstance(layer, nn.Linear):
            if layer.in_features < layer.out_features:
                print("[FAILED] Кількість нейронів зростає: ", layer.in_features, "->", layer.out_features)
                return

    print("[OK] Всі функції активації вірні")

In [26]:
def main_check():
    print("Розпочинаю перевірку моделі...")
    check_layers(model)
    check_activation_functions(model)

main_check()

Розпочинаю перевірку моделі...
[FAILED] Кількість шарів:  9
[OK] Кількість вхідних нейронів першого шару відповідає 5
[OK] Всі функції активації вірні


# Завдання 4
Визначте функцію втрат та оптимізатор

In [27]:
loss_fn = nn.MSELoss()      # Loss MSE function for regression

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)   # Optimiser (SGD) to update weights

# Завдання 5
Натренуйте нейромережу. візміть невелику кількість епох

In [41]:
import numpy as np

num_epochs = 10
losses = []

for epoch in range(num_epochs):
  losses_ = []
  for X_batch, y_batch in dataloader:
        # direct propagation
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        losses_.append(loss.item())

        # backpropagation
        loss.backward() # check gradients
        optimizer.step()
        optimizer.zero_grad()

  losses.append(np.mean(losses_))

  if epoch % 1 == 0:
          print(f'loss={losses[-1]}')

loss=0.03696931597022783
loss=0.036933298355766704
loss=0.03694808979829153
loss=0.03690842539072037
loss=0.036858037912419865
loss=0.036926676031379474
loss=0.03688721012856279
loss=0.03691241703927517
loss=0.037037329437832035
loss=0.03688539183210759


# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [31]:
torch.save(model, 'model.pt')

In [32]:
load_model = torch.load('model.pt')

<ipython-input-32-2cff9b381423>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model = torch.load('model.pt')


In [33]:
X.iloc[:10]

,age,sex,bmi,children,smoker
0,0.021739,1,0.320755,0,1
1,0.000000,0,0.479784,1,0
2,0.217391,0,0.458221,3,0
3,0.326087,0,0.180593,0,0
4,0.304348,0,0.347709,0,0
5,0.282609,1,0.261456,0,0
6,0.608696,1,0.469003,1,0
7,0.413043,1,0.315364,3,0
8,0.413043,0,0.371968,2,0
9,0.913043,1,0.264151,0,0


In [35]:
X_data = torch.FloatTensor(X.iloc[:1].values)
X_data = X_data.to(device)

X_data

tensor([[0.0217, 1.0000, 0.3208, 0.0000, 1.0000]])

In [37]:
load_model(X_data)    # with gradient

tensor([[0.1728]], grad_fn=<AddmmBackward0>)

In [38]:
with torch.no_grad():     # without gradient
  print(load_model(X_data))

tensor([[0.1728]])
